In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install sklearn
import pandas as pd
from google.colab import drive
from konlpy.tag import Mecab
import warnings
drive.mount('/content/gdrive')

In [ ]:
df_train = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/NLP텍스트분석/멜론 장르 종합 100 라벨링.xlsx", keep_default_na =False)


In [ ]:
tokenizer = Mecab()
from sklearn.feature_extraction.text import TfidfVectorizer
### 필요없는 텍스트 지움(영문)
def mecab_tokenizer(text):
  return tokenizer.morphs(text)

In [ ]:
tfidf =TfidfVectorizer(tokenizer=mecab_tokenizer,
                      ngram_range=(1,2), min_df=3, max_df=0.9)



In [ ]:
tfidf.fit(df_train["가사"].apply(lambda x : x.replace("사랑", "").replace('우리','').replace('지금','').replace('그대','')))
# 장르 판단에 영향을 주는 단어 삭제

In [ ]:
df_train_tfidf = tfidf.transform(df_train["가사"])
df_train_tfidf

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(df_train_tfidf, df_train.label)
## 벡터화
svm.score(df_train_tfidf, df_train.label)

In [ ]:
##########노래 전체 변환#######

from pathlib import Path
txt = Path('/content/gdrive/My Drive/Colab Notebooks/NLP텍스트분석/눈물비_홍진영_트로트.txt').read_text()
txt = txt.replace('\n', '')
df_test2 = [''.join(txt)]
df_test2

In [ ]:
# 장르 예측

''' 
 라벨링 번호
 발라드:0
 댄스:1
 랩힙합:2
 알앤비:3
 인디:4
 트로트:5
'''
q = tfidf.transform(df_test2)
qpd = svm.predict(q)
qpd